In [137]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [128]:
from sklearn.datasets import make_blobs
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.datasets import load_diabetes

In [138]:
# X,y = make_blobs(1000,10,centers=2,random_state=0)
# X,y = make_circles(1000,noise=0.02)
# X,y = make_moons(1000,noise=0.05)
data = pd.read_csv('diabetes.csv')

In [139]:
# plt.scatter(X[:,0],X[:,1],c = y)
# plt.show()

In [145]:
X,y = data.drop(['Outcome'],axis = 1).values,data['Outcome'].values.reshape((-1,1))

In [146]:
X.shape,y.shape

((768, 8), (768, 1))

In [163]:
class MLP:
    def __init__(self,InputSize : int, hiddenLayers: list, OutputSize:int):
        self.weights = {}

        self.weights["W1"] = np.zeros((InputSize,hiddenLayers[0]))
        self.weights["B1"] = np.zeros(hiddenLayers[0])

        wn = 2
        for i in range(1,len(hiddenLayers)):
            self.weights["W"+str(wn)] = np.zeros((hiddenLayers[i-1],hiddenLayers[i]))
            self.weights["B"+str(wn)] = np.zeros(hiddenLayers[i])

            wn += 1

        self.weights["W"+str(wn)] = np.zeros((hiddenLayers[-1],OutputSize))
        self.weights["B"+str(wn)] = np.zeros(OutputSize)

    def fit(self,X,y,lr = 0.01,epochs = 100):
        self.backwardProp(X,y,lr,epochs)

    def predict(self,X):
        ypred = self.forward(X)
        # print(ypred)
        ypred[ypred >= 0.5] = 1
        ypred[ypred < 0.5] = 0
        return ypred

    def activation(self,Z):
        return 1/(1+np.exp(-Z))

    def derA(self,Z):
        return Z*(1-Z)

    def forward(self,X):

        self.outputs = []
        A = X
        for i in range(1,len(self.weights)//2 + 1):
            Z = A@self.weights["W"+str(i)] + self.weights["B"+str(i)]
            A = self.activation(Z)
            self.outputs.append(A)
        return A

    def backwardProp(self,X,y,lr = 0.1,epochs = 1000):
        
        for e in range(epochs):
            ypred = self.forward(X)

            #Last Layer Grads
            delta = [ypred-y]
            gradw = []

            nw = len(self.weights)//2
            for o in range(len(self.outputs)-2,-1,-1):
            
                gradw.append((self.outputs[o].T)@delta[-1])
                delta.append((delta[-1]@self.weights['W'+str(nw)].T)*self.derA(self.outputs[o]))
                nw -= 1
                
            gradw.append((X.T)@delta[-1])


            nw = len(self.weights)//2
            for i in range(len(delta)):
                self.weights["W"+str(nw)] = self.weights["W"+str(nw)] - lr*gradw[i]
                self.weights["B"+str(nw)] = self.weights["B"+str(nw)] - lr*np.mean(delta[i],axis = 0)
                nw -= 1
            

In [167]:
clf = MLP(X.shape[1],[64,32],1)

In [170]:
clf.fit(X,y,0.1,1000)
ypred = clf.predict(X)

In [171]:
np.mean(ypred.reshape(-1) == y)

np.float64(0.6510416666666666)